<a href="https://colab.research.google.com/github/Rehab-04/ASU/blob/main/Copy_of_H_W1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time

In [2]:
base_url = 'https://www.careopinion.org.uk/opinions/?page={}'
def scrap_page(url):
    response = requests.get(url)
    response.raise_for_status()
    soup = BeautifulSoup(response.content, 'lxml')
    about_link = soup.find_all('a', class_='font-c-1 tooltip')
    story_titles = []
    story_opinion = []
    #list
    story_hospitals = []
    story_dates = []
    story_comments = []
    story_summaries = []
    base_domain = 'https://www.careopinion.org.uk'
    for link in about_link:
        title = link.get_text(strip=True).replace('"', '')
        story_titles.append(title)
        href = link.get('href')
        if href:
            full_url = base_domain + href
            opinion_response = requests.get(full_url)
            opinion_response.raise_for_status()
            opinion_soup = BeautifulSoup(opinion_response.content, 'lxml')


            hospital_tag = opinion_soup.find('span', class_='location-name')
            hospital = hospital_tag.get_text(strip=True) if hospital_tag else 'Not there!'
            story_hospitals.append(hospital)
#==============================================================================================================
            date_tag = opinion_soup.find('span', class_='date')
            date = date_tag.get_text(strip=True) if date_tag else 'Not there!'
            story_dates.append(date)
#==============================================================================================================
            opinion_tag = opinion_soup.find('blockquote', id='opinion_body')
            o_text = opinion_tag.get_text(strip=True) if opinion_tag else 'Not there!'
            story_opinion.append(o_text)
#==============================================================================================================
            comment_tag = opinion_soup.find('span', class_='count')
            comments = comment_tag.get_text(strip=True) if comment_tag else 'Not there!'
            story_comments.append(comments)
#==============================================================================================================
            summary_sections = opinion_soup.find_all('div', class_='feedback-section')
            summary = []
            for section in summary_sections:
                question = section.find('h3')
                answer = section.find('p')
                if question and answer:
                    summary.append(f"{question.get_text(strip=True)}: {answer.get_text(strip=True)}")
            summary_text = ' | '.join(summary) if summary else 'Not there!'
            story_summaries.append(summary_text)
            time.sleep(2)
    return story_titles, story_opinion, story_hospitals, story_dates, story_comments, story_summaries
def scrape_all_pages(base_url, num_pages):
    all_titles = []
    all_opinions = []
    all_hospitals = []
    all_dates = []
    all_comments = []
    all_summaries = []

    for page in range(1, num_pages + 1):
        print(f'page num ={page}')
        url = base_url.format(page)
        titles, opinions, hospitals, dates, comments, summaries = scrap_page(url)
        all_titles.extend(titles)
        all_opinions.extend(opinions)
        all_hospitals.extend(hospitals)
        all_dates.extend(dates)
        all_comments.extend(comments)
        all_summaries.extend(summaries)
    return all_titles, all_opinions, all_hospitals, all_dates, all_comments, all_summaries
titles, opinions, hospitals, dates, comments, summaries = scrape_all_pages(base_url, 3)
df = pd.DataFrame({
    'Title': titles,
    'Opinion': opinions,
    'Hospital Name': hospitals,
    'Date': dates,
    'Number of Comments': comments,
    'Story Summary': summaries
})




df.to_csv('care_opinion_feedback.csv', index=False)
print('finish')


page num =1
page num =2
page num =3
finish
